In [1]:
import os
import nbformat
import similar

In [2]:
def isComment(line):
    stripped = line.strip()
    if stripped and stripped[0] == '#':
        return True
    return False


def isMagic(line):
    stripped = line.strip()
    if stripped and stripped[0] == '%':
        return True
    return False


def sanitize(txt):
    lines = txt.split('\n')
    okLines = [l for l in lines if not isComment(l)]
    okLines = [l for l in okLines if not isMagic(l)]
    return '\n'.join(okLines)


def getSourceWithoutTests(filename):
    nb = nbformat.read(filename, 4)
    allCode = ''
    for c in nb.cells:
        if c['cell_type'] == 'code':
            if 'nbgrader' in c['metadata'].keys():
                if c['metadata'].get('editable', True) == False:
                    if c['metadata']['nbgrader'].get('locked', False) == True:
                        # this is a test cell, remove
                        continue

            # print(c)
            allCode += '\n'
            allCode += sanitize(c['source'])

    return allCode


In [3]:
def compare_two_files(ref, candidate):

#     比较两个文件，并返回结果

    ref_code = getSourceWithoutTests(ref)
    candidate_code = getSourceWithoutTests(candidate)
    try:
        results = similar.detect([ref_code, candidate_code])

        sum_percent = sum(func_diff_info.plagiarism_percent for func_diff_info in results[0][1])
#         sum_total_count = sum(func_diff_info.total_count for func_diff_info in results[0][1])
#         sum_plagiarism_count = sum(func_diff_info.plagiarism_count for func_diff_info in results[0][1])

        # return sum_plagiarism_count / float(sum_total_count) * 100
#         print(len(results[0][1]))
        return sum_percent / len(results[0][1]) * 100


    except similar.NoFuncException as ex:
        print('error: can not find functions from {}.'.format(ex.source))
        return 0


In [4]:
def generate_str(ref, candidate, percent):

#     通过模板，生成HTML结果文件中的内容

    li_template = '''<li style="color:{0};"><span>reference file : {1}&nbsp;</span><span>candidate file : {2}&nbsp;</span><span>&nbsp;{3}%</span></li></br>'''
    if percent >= 70:
        color = 'red'
    elif percent >= 30:
        color = 'blue'
    else:
        color = 'black'
    return li_template.format(color, ref, candidate, round(percent, 2))


In [5]:
if __name__ == '__main__':
    # 用来生成结果文件的模板
    html_template = '''
    <!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Plagiarism Detection</title>
</head>
<body>
<h1 style="text-align: center;">Plagiarism Detection</h1>
<ul>
  {0}
</ul>
</body>
</html>
'''
    li_template = '''<li style="color:{0};"><span>{1}&nbsp;</span><span>{2}&nbsp;</span><span>&nbsp;{3}%</span></li>'''
    # 目前所有的代码放这个目录下
    path = './data/Assignment 8/RandomWalk/last/'

    # 读取所有的文件目录
    filenames = []
    for filename in os.listdir(path):
        file_path = os.path.join(path, filename)
        if file_path.endswith('ipynb'):
            filenames.append(file_path)

    li_str = ''
    results = []
    for i in range(len(filenames)):
        for j in range(i + 1, len(filenames)):
            ref = filenames[i]
            candidate = filenames[j]
            # 计算ab相似度
            percent1 = compare_two_files(ref, candidate)
            # 计算ba相似度
            percent2 = compare_two_files(candidate, ref)

            results.append([ref, candidate, percent1])
            results.append([candidate, ref, percent2])

    # 排序
    sorted_results = reversed(sorted(results, key=lambda x: x[-1]))
    # 生成html内容
    for result in sorted_results:
        li_str += generate_str(result[0], result[1], result[2])
    # 将结果写入html
    with open('比较结果3.html', 'w') as f:
        f.write(html_template.format(li_str))



IndentationError: expected an indented block (<unknown>, line 45)